# Notebook for ANNDL - Homework 2

Team: All Is Well

Team members: Fatma Hamila, Kodai Takigawa, Zheng Maria Yu

In [ ]:
!unzip /dataset.zip 

# Setup
Firstly, we need to setup the environment by importing the required libraries, setting the current path (eventually mounting Google Drive), and fixing the random seed for our experiments' reproducibility.

The dataset should be present in the same folder of this notebook.

In [147]:
# Import libraries
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns

tfk = tf.keras
tfkl = tf.keras.layers

In [148]:
use_drive = False

In [149]:
# Connect to Google Drive
# Set path

if(use_drive):
    from google.colab import drive
    drive.mount('/gdrive')
    path = "/gdrive/My Drive/ANNDL/2022-23/Homework2"
    %cd /gdrive/My Drive/ANNDL/2022-23/Homework1
else:
    path = os.getcwd()
    print(str(path))

C:\Users\zheng\Documents\Uni\Magistrale\ANNDL\22-23\Homework2\Env


In [150]:
# Fix random seed for reproducibility
seed = 23

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [151]:
# Set dataset path
dataset_dir = 'dataset'

In [152]:
# Load dataset files
X_all_raw = np.load(os.path.join(dataset_dir, 'x_train.npy'))
y_all_raw = np.load(os.path.join(dataset_dir, 'y_train.npy'))

In [153]:
print(X_all_raw.shape,y_all_raw.shape)

(2429, 36, 6) (2429,)


In [8]:
max(y_all_raw),min(y_all_raw)

(11, 0)

# Data preprocessing & splitting
The extracted data is preprocessed and splitted for the K-Fold cross validation.

In [52]:
# Combine data sequences belonging to the same class
def combine_frame(x,y):
    new_data=[]
    for i in range(12):
        index=np.where(y==i)[0]
        data=x[index[0]]
        for j in index[1:]:
            data=np.concatenate((data,x[j]),axis=0)
        new_data.append(data)
    return new_data

data=combine_frame(X_all_raw,y_all_raw)
for a in data:
    print(a.shape)

(1224, 6)
(4428, 6)
(9720, 6)
(13716, 6)
(2232, 6)
(5508, 6)
(11268, 6)
(2448, 6)
(4320, 6)
(27972, 6)
(2772, 6)
(1836, 6)


In [53]:
X_train_KF = []
X_val_KF = []
y_train_KF = []
y_val_KF = []

In [54]:
# Prepare data for the K-Fold cross validation
X_train_KF.append([d[int(len(d)*0.2):] for d in data])
X_val_KF.append([d[:int(len(d)*0.2)] for d in data])

In [55]:
for i in range(3):
    fold = []
    fold.append([d[:int(len(d)*(0.2+0.2*i))] for d in data])
    fold.append([d[int(len(d)*(0.4+0.2*i)):] for d in data])
    X_train_KF.append(fold)
    X_val_KF.append([d[int(len(d)*(0.2+0.2*i)):int(len(d)*(0.4+0.2*i))] for d in data])

In [56]:
X_train_KF.append([d[:int(len(d)*0.8)] for d in data])
X_val_KF.append([d[int(len(d)*0.8):] for d in data])

In [58]:
# Split the data into shorter sequences with the sliding window
def split_data(x,y,window=24,step=1):
    data=[]
    target=[]
    
    for i in range(len(x)):
        a=x[i]
        b=y[i]
        for j in range(0,len(a)-window,step):
            data.append(a[j:j+window])
            target.append(b)
            
    return np.asarray(data),np.asarray(target)

In [59]:
# Prepare the training and the validation sets
for i in range(5):
    X_val_KF[i],y_val=split_data(X_val_KF[i],range(12),window=36,step=36)
    y_val_KF.append(y_val)
    
    if((i<1) or (i>3)):
        X_train_KF[i],y_train=split_data(X_train_KF[i],range(12),window=36,step=6)
        y_train_KF.append(y_train)
    else:
        X_temp = []
        y_temp = []
        for j in range(2):
            X_train, y_train = split_data(X_train_KF[i][j],range(12),window=36,step=6)
            X_temp.append(X_train)
            y_temp.append(y_train)
        X_train_KF[i]=np.concatenate((X_temp[0],X_temp[1]),axis=0)
        y_train_KF.append(np.concatenate((y_temp[0],y_temp[1]),axis=0))

In [60]:
# Normalize the data
def normalize_data(x):
    new_x=[]
    
    for i in range(len(x)):
        a=x[i]
        max_,min_,mean_,std_=a.max(axis=0),a.min(axis=0),a.mean(axis=0),a.std(axis=0)
        b=np.concatenate((max_,min_,mean_,std_)).reshape((-1,6))
        a=2*(a-min_)/(max_-min_)-1
        a=np.concatenate((a,b))
        new_x.append(a)
        
    return np.array(new_x)

In [61]:
for i in range(5):
    X_train_KF[i] = normalize_data(X_train_KF[i])
    X_val_KF[i] = normalize_data(X_val_KF[i])
    y_train_KF[i] = tfk.utils.to_categorical(y_train_KF[i].reshape((-1,1)))
    y_val_KF[i] = tfk.utils.to_categorical(y_val_KF[i].reshape((-1,1)))

In [62]:
X_train_KF[1].shape, y_train_KF[1].shape, X_val_KF[1].shape, y_val_KF[1].shape

((11525, 40, 6), (11525, 12), (479, 40, 6), (479, 12))

# Model training

In [63]:
input_shape = (36, 6)
classes = 12
batch_size = 128
epochs = 100

In [64]:
input_shape, classes

((36, 6), 12)

In [28]:
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout, Flatten
from keras.layers import Input, Dense, LSTM, CuDNNLSTM, concatenate, Activation, GRU, SimpleRNN,MaxPool1D,Bidirectional,GlobalMaxPooling1D,Concatenate
from attention import Attention

def build_model(input_shape, classes):

    input = tfkl.Input(shape=input_shape, name='Input')
    x_input=input

    # default 512
    x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
        
    # default 384
    l1 = Bidirectional(LSTM(units=384, return_sequences=True))(x1)
    l2 = Bidirectional(LSTM(units=384, return_sequences=True))(x_input)

    c1 = Concatenate(axis=2)((l1,l2))
    
    # default 256
    l3 = Bidirectional(LSTM(units=256, return_sequences=True))(c1)
    l4 = Bidirectional(LSTM(units=256, return_sequences=True))(l2)

    c2 = Concatenate(axis=2)((l3,l4))
    
    l6 = GlobalMaxPooling1D()(c2)
    
#    a1 = Attention()(c2)
    
    l7 = Dense(units=128, activation='selu')(l6)
    l8 = Dropout(0.05, seed=seed)(l7)
        
    output = Dense(12, activation='sigmoid')(l8)

    model = tfk.Model(inputs=input, outputs=output)
    
    # Compile the model
    model.compile(
        loss=tfk.losses.CategoricalCrossentropy(),
        optimizer=tfk.optimizers.Adam(learning_rate=1e-4),
        metrics=['accuracy']
    )

    # Return the model
    return model

In [29]:
# model = build_model((36,6), classes)
# model.summary()

# Checkpoint and Early stopping settings
Early stopping options can be activated to limit model overfitting.

There is possibility to save checkpoints during the training, in order to keep track of the performance and to get more choices of the final model.


In [30]:
os.getcwd()

'C:\\Users\\zheng\\Documents\\Uni\\Magistrale\\ANNDL\\22-23\\Homework2\\Env'

In [33]:
from datetime import datetime
def create_folders_and_callbacks(model_name):
    
    exps_dir = os.path.join('experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')
    print(now)

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    ckpt_dir = os.path.join(exp_dir, 'base_ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True,# True to save only the best epoch
                                                     monitor='val_accuracy')
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)


#     def scheduler(epoch):
#         if epoch < 30:
#             return 1e-4
#         elif epoch < 300:
#             return 1e-3
#         elif epoch < 500:
#             return 3e-4
#         else:
#             return 1e-4
        
#     sc_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    
#     callbacks.append(sc_callback)
    

    return callbacks, now

In [34]:
foldermode = "ResLSTM"

In [33]:
# callbacks, date = create_folders_and_callbacks(foldermode)

In [34]:
# foldername = foldermode + '_' + str(date)
# print(foldername)

In [27]:
# Build generator to obtain augmented data
from tensorflow.keras.utils import Sequence
import augmentationrepo.utils.augmentation as aug

class AugmentDataGenerator(Sequence):

    def __init__(self, data,target,p=0.1,batch_size=64):
        self.data = data
        self.target=target
        self.p=p
        self.augment=[aug.jitter,aug.scaling,aug.permutation,
                      aug.magnitude_warp,aug.time_warp,
                      aug.rotation,aug.window_slice,aug.window_warp]

        self.batch_size=batch_size
        self.perm=np.random.permutation(len(self.data))
        
    def __len__(self):
          if len(self.data)%self.batch_size==0:
            return len(self.data)//self.batch_size
          return len(self.data)//self.batch_size+1

    def getitem(self,x):
        data,target=self.data[x],self.target[x]
        max_,min_=data[36],data[37]
        data=data[:36]
        for f in self.augment:
            if np.random.rand()<self.p:
                data = f(np.array([data]))[0]
                
        data=(data+1)*(max_-min_)/2+min_
        return (np.array(data), np.array(target))
    
    def __getitem__(self, x):
        data,target=[],[]
        for i in self.perm[x*self.batch_size:min(len(self.data),(x+1)*self.batch_size)]:
              a,b=self.getitem(i)
              data.append(a)
              target.append(b)
        if ((x+1)*self.batch_size)>len(self.data):
            self.perm=np.random.permutation(len(self.data))
        return (np.array(data), np.array(target))
        


# generator=AugmentDataGenerator(X_train,y_train,p=0.5)
# data,target=generator.__getitem__(0)
# print(data.shape,target.shape)

In [36]:
# val_generator=AugmentDataGenerator(X_val,y_val,p=0)
# data,target=generator.__getitem__(0)
# print(data.shape,target.shape)

In [37]:
# Prepare KFold models
model = build_model((36,6), classes)
generator=AugmentDataGenerator(X_train_KF[0],y_train_KF[0],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[0],y_val_KF[0],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_02-56-07
ResLSTM_Dec17_02-56-07


In [38]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 500,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/500
181/182 [============================>.] - ETA: 0s - loss: 1.5726 - accuracy: 0.4694

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_02-56-07\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_02-56-07\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 100s 475ms/step - loss: 1.5726 - accuracy: 0.4693 - val_loss: 1.1934 - val_accuracy: 0.5908
Epoch 2/500
181/182 [============================>.] - ETA: 0s - loss: 1.3553 - accuracy: 0.5334

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_02-56-07\base_ckpts\cp_02.ckpt\assets


...

182/182 [==============================] - 84s 461ms/step - loss: 0.6031 - accuracy: 0.7890 - val_loss: 0.8293 - val_accuracy: 0.7954
Epoch 233/500
182/182 [==============================] - 15s 78ms/step - loss: 0.5963 - accuracy: 0.7905 - val_loss: 0.8833 - val_accuracy: 0.7766
Epoch 234/500
182/182 [==============================] - 14s 77ms/step - loss: 0.5941 - accuracy: 0.7905 - val_loss: 0.8166 - val_accuracy: 0.7787
Epoch 235/500
182/182 [==============================] - 14s 78ms/step - loss: 0.6028 - accuracy: 0.7849 - val_loss: 0.8677 - val_accuracy: 0.7620
Epoch 236/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5923 - accuracy: 0.7876 - val_loss: 0.8421 - val_accuracy: 0.7641
Epoch 237/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5880 - accuracy: 0.7928 - val_loss: 0.8481 - val_accuracy: 0.7578
Epoch 238/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5862 - accuracy: 0.7905 - val_loss: 0.8389 - val_a

Epoch 288/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5337 - accuracy: 0.8110 - val_loss: 0.8787 - val_accuracy: 0.7683
Epoch 289/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5413 - accuracy: 0.8062 - val_loss: 0.9114 - val_accuracy: 0.7704
Epoch 290/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5565 - accuracy: 0.8049 - val_loss: 0.9058 - val_accuracy: 0.7620
Epoch 291/500
182/182 [==============================] - 14s 79ms/step - loss: 0.5637 - accuracy: 0.7990 - val_loss: 0.9346 - val_accuracy: 0.7662
Epoch 292/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5591 - accuracy: 0.8008 - val_loss: 0.8801 - val_accuracy: 0.7537
Epoch 293/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5524 - accuracy: 0.8070 - val_loss: 0.9279 - val_accuracy: 0.7495
Epoch 294/500
182/182 [==============================] - 14s 79ms/step - loss: 0.5535 - accuracy: 0.8064 - val_loss: 0

Epoch 344/500
182/182 [==============================] - 14s 79ms/step - loss: 0.5197 - accuracy: 0.8136 - val_loss: 0.9135 - val_accuracy: 0.7724
Epoch 345/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5172 - accuracy: 0.8179 - val_loss: 0.8738 - val_accuracy: 0.7766
Epoch 346/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5235 - accuracy: 0.8124 - val_loss: 0.8684 - val_accuracy: 0.7662
Epoch 347/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5040 - accuracy: 0.8202 - val_loss: 0.8556 - val_accuracy: 0.7599
Epoch 348/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5344 - accuracy: 0.8131 - val_loss: 0.8684 - val_accuracy: 0.7641
Epoch 349/500
182/182 [==============================] - 14s 79ms/step - loss: 0.5322 - accuracy: 0.8146 - val_loss: 0.9133 - val_accuracy: 0.7599
Epoch 350/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5209 - accuracy: 0.8170 - val_loss: 0

Epoch 400/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5073 - accuracy: 0.8205 - val_loss: 1.0149 - val_accuracy: 0.7557
Epoch 401/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4964 - accuracy: 0.8245 - val_loss: 0.9491 - val_accuracy: 0.7557
Epoch 402/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5043 - accuracy: 0.8196 - val_loss: 0.9246 - val_accuracy: 0.7620
Epoch 403/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4871 - accuracy: 0.8301 - val_loss: 0.9331 - val_accuracy: 0.7516
Epoch 404/500
182/182 [==============================] - 14s 78ms/step - loss: 0.5083 - accuracy: 0.8201 - val_loss: 1.0029 - val_accuracy: 0.7474
Epoch 405/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4999 - accuracy: 0.8197 - val_loss: 0.9414 - val_accuracy: 0.7599
Epoch 406/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4929 - accuracy: 0.8264 - val_loss: 0

Epoch 456/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4835 - accuracy: 0.8282 - val_loss: 0.9840 - val_accuracy: 0.7516
Epoch 457/500
182/182 [==============================] - 14s 79ms/step - loss: 0.4685 - accuracy: 0.8357 - val_loss: 0.9781 - val_accuracy: 0.7495
Epoch 458/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4731 - accuracy: 0.8345 - val_loss: 1.0146 - val_accuracy: 0.7390
Epoch 459/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4729 - accuracy: 0.8300 - val_loss: 1.0266 - val_accuracy: 0.7390
Epoch 460/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4696 - accuracy: 0.8341 - val_loss: 1.0571 - val_accuracy: 0.7370
Epoch 461/500
182/182 [==============================] - 14s 78ms/step - loss: 0.4811 - accuracy: 0.8270 - val_loss: 1.0151 - val_accuracy: 0.7557
Epoch 462/500
182/182 [==============================] - 14s 79ms/step - loss: 0.4942 - accuracy: 0.8241 - val_loss: 0

In [39]:
model = build_model((36,6), classes)
generator=AugmentDataGenerator(X_train_KF[1],y_train_KF[1],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[1],y_val_KF[1],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_05-18-07
ResLSTM_Dec17_05-18-07


In [40]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 500,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/500
180/181 [============================>.] - ETA: 0s - loss: 1.5615 - accuracy: 0.4714

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_05-18-07\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_05-18-07\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 100s 488ms/step - loss: 1.5613 - accuracy: 0.4716 - val_loss: 1.2459 - val_accuracy: 0.6117
Epoch 2/500
181/181 [==============================] - 14s 78ms/step - loss: 1.3407 - accuracy: 0.5363 - val_loss: 1.1684 - val_accuracy: 0.5992
Epoch 3/500
180/181 [============================>.] - ETA: 0s - loss: 1.2643 - accuracy: 0.5595

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_05-18-07\base_ckpts\cp_03.ckpt\assets


...

Epoch 255/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5831 - accuracy: 0.7917 - val_loss: 1.1455 - val_accuracy: 0.6973
Epoch 256/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5651 - accuracy: 0.7984 - val_loss: 1.1166 - val_accuracy: 0.7015
Epoch 257/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5725 - accuracy: 0.7955 - val_loss: 1.1222 - val_accuracy: 0.7098
Epoch 258/500
181/181 [==============================] - 14s 80ms/step - loss: 0.5735 - accuracy: 0.7959 - val_loss: 1.1247 - val_accuracy: 0.7119
Epoch 259/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5734 - accuracy: 0.7924 - val_loss: 1.0592 - val_accuracy: 0.7098
Epoch 260/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5690 - accuracy: 0.7948 - val_loss: 1.0836 - val_accuracy: 0.7203
Epoch 261/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5690 - accuracy: 0.7993 - val_loss: 1

Epoch 311/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5365 - accuracy: 0.8091 - val_loss: 1.0790 - val_accuracy: 0.7098
Epoch 312/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5359 - accuracy: 0.8086 - val_loss: 1.0743 - val_accuracy: 0.7077
Epoch 313/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5456 - accuracy: 0.8066 - val_loss: 1.1087 - val_accuracy: 0.7015
Epoch 314/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5335 - accuracy: 0.8108 - val_loss: 1.1293 - val_accuracy: 0.6931
Epoch 315/500
181/181 [==============================] - 14s 80ms/step - loss: 0.5275 - accuracy: 0.8128 - val_loss: 1.0403 - val_accuracy: 0.7098
Epoch 316/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5324 - accuracy: 0.8106 - val_loss: 1.1047 - val_accuracy: 0.7098
Epoch 317/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5322 - accuracy: 0.8119 - val_loss: 1

Epoch 367/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5080 - accuracy: 0.8168 - val_loss: 1.0752 - val_accuracy: 0.7161
Epoch 368/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5150 - accuracy: 0.8155 - val_loss: 1.0497 - val_accuracy: 0.7056
Epoch 369/500
181/181 [==============================] - 14s 80ms/step - loss: 0.5063 - accuracy: 0.8223 - val_loss: 1.1148 - val_accuracy: 0.7140
Epoch 370/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5163 - accuracy: 0.8117 - val_loss: 1.0887 - val_accuracy: 0.7140
Epoch 371/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5024 - accuracy: 0.8204 - val_loss: 1.1159 - val_accuracy: 0.6952
Epoch 372/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5092 - accuracy: 0.8174 - val_loss: 1.1799 - val_accuracy: 0.6952
Epoch 373/500
181/181 [==============================] - 14s 79ms/step - loss: 0.5008 - accuracy: 0.8227 - val_loss: 1

Epoch 423/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4929 - accuracy: 0.8272 - val_loss: 1.1707 - val_accuracy: 0.6910
Epoch 424/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4814 - accuracy: 0.8281 - val_loss: 1.2310 - val_accuracy: 0.6973
Epoch 425/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4841 - accuracy: 0.8295 - val_loss: 1.1887 - val_accuracy: 0.7035
Epoch 426/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4760 - accuracy: 0.8306 - val_loss: 1.2170 - val_accuracy: 0.7056
Epoch 427/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4855 - accuracy: 0.8309 - val_loss: 1.1887 - val_accuracy: 0.7223
Epoch 428/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4753 - accuracy: 0.8338 - val_loss: 1.1064 - val_accuracy: 0.7307
Epoch 429/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4840 - accuracy: 0.8283 - val_loss: 1

Epoch 479/500
181/181 [==============================] - 14s 80ms/step - loss: 0.4516 - accuracy: 0.8366 - val_loss: 1.2141 - val_accuracy: 0.7098
Epoch 480/500
181/181 [==============================] - 15s 80ms/step - loss: 0.4709 - accuracy: 0.8347 - val_loss: 1.1595 - val_accuracy: 0.7182
Epoch 481/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4617 - accuracy: 0.8396 - val_loss: 1.1522 - val_accuracy: 0.7161
Epoch 482/500
181/181 [==============================] - 15s 80ms/step - loss: 0.4642 - accuracy: 0.8340 - val_loss: 1.0899 - val_accuracy: 0.7286
Epoch 483/500
181/181 [==============================] - 14s 79ms/step - loss: 0.4646 - accuracy: 0.8331 - val_loss: 1.1773 - val_accuracy: 0.7203
Epoch 484/500
181/181 [==============================] - 14s 80ms/step - loss: 0.4669 - accuracy: 0.8352 - val_loss: 1.1238 - val_accuracy: 0.7098
Epoch 485/500
181/181 [==============================] - 14s 80ms/step - loss: 0.4619 - accuracy: 0.8387 - val_loss: 1

In [41]:
model = build_model((36,6), classes)
generator=AugmentDataGenerator(X_train_KF[2],y_train_KF[2],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[2],y_val_KF[2],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_07-38-45
ResLSTM_Dec17_07-38-45


In [42]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 500,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/500
180/181 [============================>.] - ETA: 0s - loss: 1.5781 - accuracy: 0.4619

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_07-38-45\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_07-38-45\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 102s 499ms/step - loss: 1.5781 - accuracy: 0.4619 - val_loss: 1.2406 - val_accuracy: 0.5783
Epoch 2/500
180/181 [============================>.] - ETA: 0s - loss: 1.3705 - accuracy: 0.5246

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_07-38-45\base_ckpts\cp_02.ckpt\assets


...

181/181 [==============================] - 86s 478ms/step - loss: 0.5883 - accuracy: 0.7933 - val_loss: 0.9923 - val_accuracy: 0.7641
Epoch 252/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5800 - accuracy: 0.7955 - val_loss: 0.9395 - val_accuracy: 0.7453
Epoch 253/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5904 - accuracy: 0.7906 - val_loss: 1.0078 - val_accuracy: 0.7349
Epoch 254/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5665 - accuracy: 0.8005 - val_loss: 1.0052 - val_accuracy: 0.7307
Epoch 255/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5864 - accuracy: 0.7905 - val_loss: 0.9313 - val_accuracy: 0.7474
Epoch 256/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5899 - accuracy: 0.7886 - val_loss: 0.9562 - val_accuracy: 0.7578
Epoch 257/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5864 - accuracy: 0.7895 - val_loss: 0.9542 - val_a

Epoch 307/500
181/181 [==============================] - 15s 82ms/step - loss: 0.5542 - accuracy: 0.8054 - val_loss: 1.0265 - val_accuracy: 0.7453
Epoch 308/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5556 - accuracy: 0.8024 - val_loss: 1.0178 - val_accuracy: 0.7495
Epoch 309/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5599 - accuracy: 0.7997 - val_loss: 1.0550 - val_accuracy: 0.7432
Epoch 310/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5542 - accuracy: 0.8026 - val_loss: 1.0797 - val_accuracy: 0.7474
Epoch 311/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5498 - accuracy: 0.8052 - val_loss: 1.0120 - val_accuracy: 0.7328
Epoch 312/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5523 - accuracy: 0.8058 - val_loss: 1.0064 - val_accuracy: 0.7557
Epoch 313/500
181/181 [==============================] - 15s 82ms/step - loss: 0.5478 - accuracy: 0.8073 - val_loss: 1

Epoch 363/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5249 - accuracy: 0.8099 - val_loss: 1.0535 - val_accuracy: 0.7056
Epoch 364/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5263 - accuracy: 0.8160 - val_loss: 1.0279 - val_accuracy: 0.7223
Epoch 365/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5330 - accuracy: 0.8148 - val_loss: 1.0407 - val_accuracy: 0.7265
Epoch 366/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5101 - accuracy: 0.8208 - val_loss: 1.0277 - val_accuracy: 0.7265
Epoch 367/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5264 - accuracy: 0.8125 - val_loss: 0.9987 - val_accuracy: 0.7203
Epoch 368/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5226 - accuracy: 0.8126 - val_loss: 0.9976 - val_accuracy: 0.7286
Epoch 369/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5210 - accuracy: 0.8161 - val_loss: 1

Epoch 419/500
181/181 [==============================] - 15s 82ms/step - loss: 0.5030 - accuracy: 0.8258 - val_loss: 1.0482 - val_accuracy: 0.7328
Epoch 420/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4943 - accuracy: 0.8272 - val_loss: 1.0637 - val_accuracy: 0.7411
Epoch 421/500
181/181 [==============================] - 15s 80ms/step - loss: 0.4916 - accuracy: 0.8235 - val_loss: 1.0640 - val_accuracy: 0.7098
Epoch 422/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5040 - accuracy: 0.8236 - val_loss: 1.0097 - val_accuracy: 0.7244
Epoch 423/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5070 - accuracy: 0.8183 - val_loss: 1.0346 - val_accuracy: 0.7140
Epoch 424/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4803 - accuracy: 0.8306 - val_loss: 0.9983 - val_accuracy: 0.7370
Epoch 425/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4867 - accuracy: 0.8266 - val_loss: 1

Epoch 475/500
181/181 [==============================] - 15s 80ms/step - loss: 0.4584 - accuracy: 0.8347 - val_loss: 1.1575 - val_accuracy: 0.7265
Epoch 476/500
181/181 [==============================] - 15s 80ms/step - loss: 0.4659 - accuracy: 0.8363 - val_loss: 1.0822 - val_accuracy: 0.7223
Epoch 477/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4823 - accuracy: 0.8294 - val_loss: 1.0529 - val_accuracy: 0.7328
Epoch 478/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4767 - accuracy: 0.8298 - val_loss: 1.1550 - val_accuracy: 0.7265
Epoch 479/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4558 - accuracy: 0.8396 - val_loss: 1.0811 - val_accuracy: 0.7474
Epoch 480/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4736 - accuracy: 0.8321 - val_loss: 1.0986 - val_accuracy: 0.7328
Epoch 481/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4728 - accuracy: 0.8321 - val_loss: 0

In [43]:
model = build_model((36,6), classes)
generator=AugmentDataGenerator(X_train_KF[3],y_train_KF[3],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[3],y_val_KF[3],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_10-02-13
ResLSTM_Dec17_10-02-13


In [44]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 500,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/500
180/181 [============================>.] - ETA: 0s - loss: 1.5592 - accuracy: 0.4691

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_10-02-13\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_10-02-13\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 102s 499ms/step - loss: 1.5589 - accuracy: 0.4692 - val_loss: 1.2433 - val_accuracy: 0.5825
Epoch 2/500
180/181 [============================>.] - ETA: 0s - loss: 1.3439 - accuracy: 0.5336

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_10-02-13\base_ckpts\cp_02.ckpt\assets


...

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_10-02-13\base_ckpts\cp_375.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_10-02-13\base_ckpts\cp_375.ckpt\assets


181/181 [==============================] - 84s 466ms/step - loss: 0.5098 - accuracy: 0.8190 - val_loss: 1.1030 - val_accuracy: 0.7432
Epoch 376/500
181/181 [==============================] - 15s 80ms/step - loss: 0.5039 - accuracy: 0.8229 - val_loss: 1.1188 - val_accuracy: 0.7203
Epoch 377/500
181/181 [==============================] - 15s 81ms/step - loss: 0.5069 - accuracy: 0.8254 - val_loss: 1.1446 - val_accuracy: 0.7161
Epoch 378/500
181/181 [==============================] - 15s 82ms/step - loss: 0.5104 - accuracy: 0.8189 - val_loss: 1.1465 - val_accuracy: 0.7015
Epoch 379/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4984 - accuracy: 0.8259 - val_loss: 1.1796 - val_accuracy: 0.6868
Epoch 380/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4959 - accuracy: 0.8248 - val_loss: 1.1826 - val_accuracy: 0.7119
Epoch 381/500
181/181 [==============================] - 15s 82ms/step - loss: 0.5069 - accuracy: 0.8187 - val_loss: 1.1590 - val_a

Epoch 431/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4845 - accuracy: 0.8285 - val_loss: 1.1551 - val_accuracy: 0.7015
Epoch 432/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4847 - accuracy: 0.8272 - val_loss: 1.1601 - val_accuracy: 0.6973
Epoch 433/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4821 - accuracy: 0.8262 - val_loss: 1.1936 - val_accuracy: 0.7203
Epoch 434/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4812 - accuracy: 0.8251 - val_loss: 1.1815 - val_accuracy: 0.7015
Epoch 435/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4876 - accuracy: 0.8290 - val_loss: 1.1763 - val_accuracy: 0.7015
Epoch 436/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4828 - accuracy: 0.8281 - val_loss: 1.1773 - val_accuracy: 0.7015
Epoch 437/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4760 - accuracy: 0.8291 - val_loss: 1

Epoch 487/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4594 - accuracy: 0.8403 - val_loss: 1.2227 - val_accuracy: 0.7119
Epoch 488/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4682 - accuracy: 0.8366 - val_loss: 1.1583 - val_accuracy: 0.7307
Epoch 489/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4599 - accuracy: 0.8397 - val_loss: 1.1924 - val_accuracy: 0.7286
Epoch 490/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4655 - accuracy: 0.8329 - val_loss: 1.1727 - val_accuracy: 0.7035
Epoch 491/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4535 - accuracy: 0.8370 - val_loss: 1.1831 - val_accuracy: 0.7077
Epoch 492/500
181/181 [==============================] - 15s 82ms/step - loss: 0.4589 - accuracy: 0.8392 - val_loss: 1.2020 - val_accuracy: 0.7182
Epoch 493/500
181/181 [==============================] - 15s 81ms/step - loss: 0.4582 - accuracy: 0.8369 - val_loss: 1

In [45]:
model = build_model((36,6), classes)
generator=AugmentDataGenerator(X_train_KF[4],y_train_KF[4],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[4],y_val_KF[4],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_12-25-29
ResLSTM_Dec17_12-25-29


In [46]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 500,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/500
181/182 [============================>.] - ETA: 0s - loss: 1.5754 - accuracy: 0.4763

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_12-25-29\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_12-25-29\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 105s 485ms/step - loss: 1.5753 - accuracy: 0.4762 - val_loss: 1.1888 - val_accuracy: 0.6013
Epoch 2/500
181/182 [============================>.] - ETA: 0s - loss: 1.3545 - accuracy: 0.5337

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_12-25-29\base_ckpts\cp_02.ckpt\assets


...

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_12-25-29\base_ckpts\cp_351.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_12-25-29\base_ckpts\cp_351.ckpt\assets


182/182 [==============================] - 84s 463ms/step - loss: 0.5156 - accuracy: 0.8162 - val_loss: 1.1072 - val_accuracy: 0.7578
Epoch 352/500
182/182 [==============================] - 15s 82ms/step - loss: 0.5211 - accuracy: 0.8150 - val_loss: 1.0759 - val_accuracy: 0.7370
Epoch 353/500
182/182 [==============================] - 15s 82ms/step - loss: 0.5142 - accuracy: 0.8150 - val_loss: 1.1735 - val_accuracy: 0.7453
Epoch 354/500
182/182 [==============================] - 15s 81ms/step - loss: 0.5088 - accuracy: 0.8190 - val_loss: 1.1214 - val_accuracy: 0.7453
Epoch 355/500
182/182 [==============================] - 15s 82ms/step - loss: 0.4982 - accuracy: 0.8201 - val_loss: 1.1770 - val_accuracy: 0.7307
Epoch 356/500
182/182 [==============================] - 15s 81ms/step - loss: 0.5100 - accuracy: 0.8182 - val_loss: 1.2075 - val_accuracy: 0.7432
Epoch 357/500
182/182 [==============================] - 15s 82ms/step - loss: 0.5060 - accuracy: 0.8208 - val_loss: 1.1573 - val_a

Epoch 407/500
182/182 [==============================] - 15s 83ms/step - loss: 0.4907 - accuracy: 0.8262 - val_loss: 1.1135 - val_accuracy: 0.7390
Epoch 408/500
182/182 [==============================] - 15s 83ms/step - loss: 0.4872 - accuracy: 0.8256 - val_loss: 1.1464 - val_accuracy: 0.7370
Epoch 409/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4902 - accuracy: 0.8295 - val_loss: 1.0856 - val_accuracy: 0.7161
Epoch 410/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4826 - accuracy: 0.8343 - val_loss: 1.1183 - val_accuracy: 0.7453
Epoch 411/500
182/182 [==============================] - 15s 83ms/step - loss: 0.4740 - accuracy: 0.8342 - val_loss: 1.1087 - val_accuracy: 0.7349
Epoch 412/500
182/182 [==============================] - 15s 83ms/step - loss: 0.4759 - accuracy: 0.8325 - val_loss: 1.1425 - val_accuracy: 0.7265
Epoch 413/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4862 - accuracy: 0.8284 - val_loss: 1

Epoch 463/500
182/182 [==============================] - 15s 85ms/step - loss: 0.4706 - accuracy: 0.8392 - val_loss: 1.2579 - val_accuracy: 0.7223
Epoch 464/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4555 - accuracy: 0.8415 - val_loss: 1.1648 - val_accuracy: 0.7182
Epoch 465/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4602 - accuracy: 0.8383 - val_loss: 1.2221 - val_accuracy: 0.7265
Epoch 466/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4553 - accuracy: 0.8373 - val_loss: 1.2480 - val_accuracy: 0.7182
Epoch 467/500
182/182 [==============================] - 15s 85ms/step - loss: 0.4753 - accuracy: 0.8332 - val_loss: 1.2443 - val_accuracy: 0.7223
Epoch 468/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4649 - accuracy: 0.8384 - val_loss: 1.1963 - val_accuracy: 0.7370
Epoch 469/500
182/182 [==============================] - 15s 84ms/step - loss: 0.4599 - accuracy: 0.8376 - val_loss: 1

In [36]:
from keras.callbacks import Callback
from keras import backend as K
import math

class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0., verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
# callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

In [30]:
def categorical_focal_loss(y_true, y_pred):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    
    alpha = [[.25, .2, .1, .25, .35, .25, .15, .35, .1, .2, .1, .3]]

    alpha = np.array(alpha, dtype=np.float32)
    
    gamma = 2.

    """
    :param y_true: A tensor of the same shape as `y_pred`
    :param y_pred: A tensor resulting from a softmax
    :return: Output tensor.
    """

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

    # Calculate Cross Entropy
    cross_entropy = -y_true * K.log(y_pred)

    # Calculate Focal Loss
    c_loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

    # Compute mean loss in mini_batch
    return K.mean(K.sum(c_loss, axis=-1))

In [57]:
model=tfk.models.load_model("experiments/KF_0.ckpt/base_ckpts/cp_232.ckpt")

In [58]:
model.compile(
            loss=categorical_focal_loss,
            optimizer=tfk.optimizers.SGD(learning_rate=1e-1),
            metrics=['accuracy']
        )

In [59]:
generator=AugmentDataGenerator(X_train_KF[0],y_train_KF[0],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[0],y_val_KF[0],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_16-22-03
ResLSTM_Dec17_16-22-03


In [60]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/100
  6/182 [..............................] - ETA: 18s - loss: 0.0023 - accuracy: 0.7969    WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0929s vs `on_train_batch_end` time: 0.4331s). Check your callbacks.


182/182 [==============================] - ETA: 0s - loss: 4.5566e-04 - accuracy: 0.7875

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-22-03\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-22-03\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 105s 508ms/step - loss: 4.5566e-04 - accuracy: 0.7875 - val_loss: 0.0015 - val_accuracy: 0.7975 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 15s 83ms/step - loss: 4.0995e-04 - accuracy: 0.7891 - val_loss: 8.7791e-04 - val_accuracy: 0.7912 - lr: 0.0999
Epoch 3/100
182/182 [==============================] - 15s 83ms/step - loss: 3.1933e-04 - accuracy: 0.7799 - val_loss: 0.0010 - val_accuracy: 0.7975 - lr: 0.0996
Epoch 4/100
182/182 [==============================] - 15s 83ms/step - loss: 2.5492e-04 - accuracy: 0.7868 - val_loss: 0.0011 - val_accuracy: 0.7975 - lr: 0.0991
Epoch 5/100
182/182 [==============================] - 15s 83ms/step - loss: 2.5364e-04 - accuracy: 0.7830 - val_loss: 0.0011 - val_accuracy: 0.7912 - lr: 0.0984
Epoch 6/100
182/182 [==============================] - 15s 83ms/step - loss: 4.4426e-04 - accuracy: 0.7790 - val_loss: 7.8739e-04 - val_accuracy: 0.7829 - lr: 0.0976
Epoch 7/100
182/182 [=========

182/182 [==============================] - 15s 83ms/step - loss: 7.2395e-05 - accuracy: 0.7327 - val_loss: 1.4855e-04 - val_accuracy: 0.7307 - lr: 0.0996
Epoch 100/100
182/182 [==============================] - 15s 84ms/step - loss: 4.7934e-05 - accuracy: 0.7421 - val_loss: 1.4890e-04 - val_accuracy: 0.7328 - lr: 0.0999


In [61]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_16-49-05
Epoch 1/100
  6/182 [..............................] - ETA: 18s - loss: 3.3865e-06 - accuracy: 0.7188WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0694s vs `on_train_batch_end` time: 0.4451s). Check your callbacks.


181/182 [============================>.] - ETA: 0s - loss: 7.6802e-05 - accuracy: 0.7357

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-49-05\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-49-05\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 88s 471ms/step - loss: 7.6750e-05 - accuracy: 0.7358 - val_loss: 1.6569e-04 - val_accuracy: 0.7307 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 15s 82ms/step - loss: 3.4944e-05 - accuracy: 0.7382 - val_loss: 1.4272e-04 - val_accuracy: 0.7307 - lr: 0.0999
Epoch 3/100
182/182 [==============================] - 15s 83ms/step - loss: 6.4477e-05 - accuracy: 0.7359 - val_loss: 1.6149e-04 - val_accuracy: 0.7307 - lr: 0.0996
Epoch 4/100
181/182 [============================>.] - ETA: 0s - loss: 2.7680e-05 - accuracy: 0.7383

...

182/182 [==============================] - 86s 475ms/step - loss: 2.7662e-05 - accuracy: 0.7384 - val_loss: 1.4698e-04 - val_accuracy: 0.7328 - lr: 0.0991
Epoch 5/100
181/182 [============================>.] - ETA: 0s - loss: 5.0509e-05 - accuracy: 0.7348

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-49-05\base_ckpts\cp_05.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_16-49-05\base_ckpts\cp_05.ckpt\assets


182/182 [==============================] - 91s 498ms/step - loss: 5.0474e-05 - accuracy: 0.7349 - val_loss: 1.4967e-04 - val_accuracy: 0.7349 - lr: 0.0984
Epoch 6/100
182/182 [==============================] - 15s 84ms/step - loss: 5.4247e-05 - accuracy: 0.7409 - val_loss: 1.2854e-04 - val_accuracy: 0.7265 - lr: 0.0976
Epoch 7/100
182/182 [==============================] - 15s 82ms/step - loss: 3.7721e-05 - accuracy: 0.7337 - val_loss: 1.3035e-04 - val_accuracy: 0.7286 - lr: 0.0965
Epoch 8/100
182/182 [==============================] - 15s 83ms/step - loss: 5.4779e-05 - accuracy: 0.7258 - val_loss: 1.2057e-04 - val_accuracy: 0.7328 - lr: 0.0952
Epoch 9/100
182/182 [==============================] - 15s 83ms/step - loss: 3.1991e-05 - accuracy: 0.7338 - val_loss: 1.2909e-04 - val_accuracy: 0.7349 - lr: 0.0938
Epoch 10/100
182/182 [==============================] - 15s 83ms/step - loss: 4.5322e-05 - accuracy: 0.7372 - val_loss: 1.1395e-04 - val_accuracy: 0.7307 - lr: 0.0922
Epoch 11/100
1

182/182 [==============================] - 15s 84ms/step - loss: 1.3861e-05 - accuracy: 0.7213 - val_loss: 7.0386e-05 - val_accuracy: 0.7161 - lr: 9.8475e-04
Epoch 55/100
182/182 [==============================] - 15s 83ms/step - loss: 2.7112e-05 - accuracy: 0.7193 - val_loss: 7.0293e-05 - val_accuracy: 0.7161 - lr: 0.0017
Epoch 56/100
182/182 [==============================] - 15s 83ms/step - loss: 3.7678e-05 - accuracy: 0.7184 - val_loss: 7.0226e-05 - val_accuracy: 0.7161 - lr: 0.0025
Epoch 57/100
182/182 [==============================] - 15s 83ms/step - loss: 1.8978e-05 - accuracy: 0.7148 - val_loss: 7.0216e-05 - val_accuracy: 0.7161 - lr: 0.0036
Epoch 58/100
182/182 [==============================] - 15s 84ms/step - loss: 9.1536e-05 - accuracy: 0.7183 - val_loss: 7.0155e-05 - val_accuracy: 0.7182 - lr: 0.0049
Epoch 59/100
182/182 [==============================] - 15s 83ms/step - loss: 7.0893e-05 - accuracy: 0.7183 - val_loss: 6.9598e-05 - val_accuracy: 0.7182 - lr: 0.0063
Epoch 6

In [62]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_17-18-05
Epoch 1/100
  6/182 [..............................] - ETA: 22s - loss: 2.6650e-05 - accuracy: 0.7448WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0926s vs `on_train_batch_end` time: 0.4422s). Check your callbacks.


181/182 [============================>.] - ETA: 0s - loss: 1.4827e-05 - accuracy: 0.7166

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-18-05\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-18-05\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 88s 474ms/step - loss: 1.4817e-05 - accuracy: 0.7167 - val_loss: 4.8686e-05 - val_accuracy: 0.7119 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 16s 84ms/step - loss: 1.5049e-05 - accuracy: 0.7159 - val_loss: 3.9718e-05 - val_accuracy: 0.7119 - lr: 0.0999
Epoch 3/100
182/182 [==============================] - 15s 85ms/step - loss: 7.7207e-05 - accuracy: 0.7070 - val_loss: 3.7989e-05 - val_accuracy: 0.7098 - lr: 0.0996
Epoch 4/100
182/182 [==============================] - 15s 85ms/step - loss: 1.5837e-05 - accuracy: 0.7034 - val_loss: 3.5361e-05 - val_accuracy: 0.7077 - lr: 0.0991
Epoch 5/100
182/182 [==============================] - 15s 84ms/step - loss: 2.0253e-05 - accuracy: 0.7009 - val_loss: 3.7852e-05 - val_accuracy: 0.7098 - lr: 0.0984
Epoch 6/100
182/182 [==============================] - 15s 84ms/step - loss: 2.4044e-05 - accuracy: 0.7074 - val_loss: 3.5771e-05 - val_accuracy: 0.7119 - lr: 0.0976
Epoch 7/100
181

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-18-05\base_ckpts\cp_07.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-18-05\base_ckpts\cp_07.ckpt\assets


182/182 [==============================] - 91s 502ms/step - loss: 2.8703e-05 - accuracy: 0.7048 - val_loss: 3.7652e-05 - val_accuracy: 0.7161 - lr: 0.0965
Epoch 8/100
182/182 [==============================] - 15s 83ms/step - loss: 1.4212e-05 - accuracy: 0.7066 - val_loss: 3.7904e-05 - val_accuracy: 0.7161 - lr: 0.0952
Epoch 9/100
182/182 [==============================] - 15s 83ms/step - loss: 1.0449e-04 - accuracy: 0.7066 - val_loss: 3.9962e-05 - val_accuracy: 0.7140 - lr: 0.0938
Epoch 10/100
182/182 [==============================] - 15s 83ms/step - loss: 3.5223e-05 - accuracy: 0.7016 - val_loss: 3.7880e-05 - val_accuracy: 0.7119 - lr: 0.0922
Epoch 11/100
182/182 [==============================] - 15s 83ms/step - loss: 2.4721e-05 - accuracy: 0.7125 - val_loss: 3.4260e-05 - val_accuracy: 0.7098 - lr: 0.0905
Epoch 12/100
182/182 [==============================] - 15s 83ms/step - loss: 1.0911e-05 - accuracy: 0.7084 - val_loss: 3.4679e-05 - val_accuracy: 0.7140 - lr: 0.0885
Epoch 13/100

Epoch 56/100
182/182 [==============================] - 15s 83ms/step - loss: 5.2712e-05 - accuracy: 0.7038 - val_loss: 2.4414e-05 - val_accuracy: 0.7077 - lr: 0.0025
Epoch 57/100
182/182 [==============================] - 15s 84ms/step - loss: 1.9710e-05 - accuracy: 0.7002 - val_loss: 2.4412e-05 - val_accuracy: 0.7077 - lr: 0.0036
Epoch 58/100
182/182 [==============================] - 15s 84ms/step - loss: 1.8891e-05 - accuracy: 0.7034 - val_loss: 2.4465e-05 - val_accuracy: 0.7077 - lr: 0.0049
Epoch 59/100
182/182 [==============================] - 15s 84ms/step - loss: 1.9862e-05 - accuracy: 0.7071 - val_loss: 2.4483e-05 - val_accuracy: 0.7077 - lr: 0.0063
Epoch 60/100
182/182 [==============================] - 15s 84ms/step - loss: 1.5984e-05 - accuracy: 0.7019 - val_loss: 2.4504e-05 - val_accuracy: 0.7077 - lr: 0.0079
Epoch 61/100
182/182 [==============================] - 15s 83ms/step - loss: 4.1654e-05 - accuracy: 0.7082 - val_loss: 2.4501e-05 - val_accuracy: 0.7077 - lr: 0.009

In [63]:
model=tfk.models.load_model("experiments/KF_1.ckpt/base_ckpts/cp_143.ckpt")

In [64]:
model.compile(
            loss=categorical_focal_loss,
            optimizer=tfk.optimizers.SGD(learning_rate=1e-1),
            metrics=['accuracy']
        )

In [65]:
generator=AugmentDataGenerator(X_train_KF[1],y_train_KF[1],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[1],y_val_KF[1],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_17-47-11
ResLSTM_Dec17_17-47-11


In [66]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/100
  6/181 [..............................] - ETA: 18s - loss: 0.0017 - accuracy: 0.7630  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0780s vs `on_train_batch_end` time: 0.4412s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 4.9939e-04 - accuracy: 0.7604

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-47-11\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_17-47-11\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 105s 513ms/step - loss: 4.9918e-04 - accuracy: 0.7604 - val_loss: 0.0040 - val_accuracy: 0.7453 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 85ms/step - loss: 3.9082e-04 - accuracy: 0.7660 - val_loss: 0.0036 - val_accuracy: 0.7307 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 16s 85ms/step - loss: 3.3421e-04 - accuracy: 0.7609 - val_loss: 0.0038 - val_accuracy: 0.7286 - lr: 0.0996
Epoch 4/100
181/181 [==============================] - 15s 85ms/step - loss: 3.0042e-04 - accuracy: 0.7511 - val_loss: 0.0032 - val_accuracy: 0.7140 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 15s 85ms/step - loss: 3.3083e-04 - accuracy: 0.7550 - val_loss: 0.0026 - val_accuracy: 0.7161 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 15s 85ms/step - loss: 2.7458e-04 - accuracy: 0.7486 - val_loss: 0.0024 - val_accuracy: 0.7119 - lr: 0.0976
Epoch 7/100
181/181 [=================

181/181 [==============================] - 16s 87ms/step - loss: 4.6447e-05 - accuracy: 0.6995 - val_loss: 1.5231e-04 - val_accuracy: 0.6472 - lr: 0.0996
Epoch 100/100
181/181 [==============================] - 16s 87ms/step - loss: 4.4659e-05 - accuracy: 0.6920 - val_loss: 1.5383e-04 - val_accuracy: 0.6472 - lr: 0.0999


In [67]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_18-14-39
Epoch 1/100
  6/181 [..............................] - ETA: 21s - loss: 4.9350e-06 - accuracy: 0.6406WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0912s vs `on_train_batch_end` time: 0.4425s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 4.5558e-05 - accuracy: 0.7009

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-14-39\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-14-39\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 90s 485ms/step - loss: 4.5538e-05 - accuracy: 0.7009 - val_loss: 1.5760e-04 - val_accuracy: 0.6514 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 85ms/step - loss: 3.8920e-05 - accuracy: 0.6985 - val_loss: 1.4905e-04 - val_accuracy: 0.6514 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 15s 85ms/step - loss: 2.0021e-05 - accuracy: 0.6991 - val_loss: 1.4358e-04 - val_accuracy: 0.6514 - lr: 0.0996
Epoch 4/100
181/181 [==============================] - 16s 86ms/step - loss: 2.5364e-05 - accuracy: 0.6989 - val_loss: 1.3274e-04 - val_accuracy: 0.6514 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 85ms/step - loss: 3.6865e-05 - accuracy: 0.6949 - val_loss: 1.2468e-04 - val_accuracy: 0.6472 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 86ms/step - loss: 3.9602e-05 - accuracy: 0.7013 - val_loss: 1.2172e-04 - val_accuracy: 0.6472 - lr: 0.0976
Epoch 7/100
181

181/181 [==============================] - 16s 86ms/step - loss: 3.6341e-05 - accuracy: 0.6868 - val_loss: 4.6712e-05 - val_accuracy: 0.6430 - lr: 0.0996
Epoch 100/100
181/181 [==============================] - 16s 86ms/step - loss: 2.7806e-05 - accuracy: 0.6865 - val_loss: 4.7148e-05 - val_accuracy: 0.6493 - lr: 0.0999


In [68]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_18-41-59
Epoch 1/100
  6/181 [..............................] - ETA: 20s - loss: 1.3241e-06 - accuracy: 0.7188WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0779s vs `on_train_batch_end` time: 0.4508s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 1.0742e-05 - accuracy: 0.6845

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-41-59\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-41-59\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 89s 479ms/step - loss: 1.0738e-05 - accuracy: 0.6843 - val_loss: 4.6131e-05 - val_accuracy: 0.6451 - lr: 0.1000
Epoch 2/100
180/181 [============================>.] - ETA: 0s - loss: 1.8743e-05 - accuracy: 0.6828

...

181/181 [==============================] - 88s 489ms/step - loss: 1.4669e-05 - accuracy: 0.6842 - val_loss: 4.2911e-05 - val_accuracy: 0.6534 - lr: 0.0996
Epoch 4/100
180/181 [============================>.] - ETA: 0s - loss: 1.4916e-05 - accuracy: 0.6803

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-41-59\base_ckpts\cp_04.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_18-41-59\base_ckpts\cp_04.ckpt\assets


181/181 [==============================] - 90s 496ms/step - loss: 1.4909e-05 - accuracy: 0.6803 - val_loss: 4.3118e-05 - val_accuracy: 0.6555 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 86ms/step - loss: 1.0377e-05 - accuracy: 0.6823 - val_loss: 4.1451e-05 - val_accuracy: 0.6534 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 86ms/step - loss: 3.0152e-05 - accuracy: 0.6884 - val_loss: 3.9097e-05 - val_accuracy: 0.6555 - lr: 0.0976
Epoch 7/100
181/181 [==============================] - 16s 86ms/step - loss: 1.5617e-05 - accuracy: 0.6858 - val_loss: 3.7964e-05 - val_accuracy: 0.6555 - lr: 0.0965
Epoch 8/100
181/181 [==============================] - 16s 87ms/step - loss: 1.0115e-05 - accuracy: 0.6827 - val_loss: 3.7188e-05 - val_accuracy: 0.6534 - lr: 0.0952
Epoch 9/100
181/181 [==============================] - 16s 87ms/step - loss: 2.4317e-05 - accuracy: 0.6828 - val_loss: 3.6694e-05 - val_accuracy: 0.6534 - lr: 0.0938
Epoch 10/100
18

181/181 [==============================] - 16s 87ms/step - loss: 2.3431e-05 - accuracy: 0.6795 - val_loss: 2.0186e-05 - val_accuracy: 0.6472 - lr: 4.9387e-04
Epoch 54/100
181/181 [==============================] - 16s 87ms/step - loss: 9.8454e-06 - accuracy: 0.6765 - val_loss: 2.0191e-05 - val_accuracy: 0.6472 - lr: 9.8475e-04
Epoch 55/100
181/181 [==============================] - 16s 86ms/step - loss: 1.1755e-05 - accuracy: 0.6761 - val_loss: 2.0198e-05 - val_accuracy: 0.6472 - lr: 0.0017
Epoch 56/100
181/181 [==============================] - 16s 87ms/step - loss: 2.6619e-05 - accuracy: 0.6797 - val_loss: 2.0209e-05 - val_accuracy: 0.6472 - lr: 0.0025
Epoch 57/100
181/181 [==============================] - 16s 86ms/step - loss: 6.8672e-06 - accuracy: 0.6757 - val_loss: 2.0144e-05 - val_accuracy: 0.6472 - lr: 0.0036
Epoch 58/100
181/181 [==============================] - 16s 87ms/step - loss: 7.4851e-06 - accuracy: 0.6775 - val_loss: 2.0145e-05 - val_accuracy: 0.6472 - lr: 0.0049
Epo

In [69]:
model=tfk.models.load_model("experiments/KF_2.ckpt/base_ckpts/cp_251.ckpt")

In [70]:
model.compile(
            loss=categorical_focal_loss,
            optimizer=tfk.optimizers.SGD(learning_rate=1e-1),
            metrics=['accuracy']
        )

In [71]:
generator=AugmentDataGenerator(X_train_KF[2],y_train_KF[2],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[2],y_val_KF[2],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_19-14-19
ResLSTM_Dec17_19-14-19


In [72]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/100
  6/181 [..............................] - ETA: 21s - loss: 1.1497e-04 - accuracy: 0.8099WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0860s vs `on_train_batch_end` time: 0.4633s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 4.9230e-04 - accuracy: 0.7890

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-14-19\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-14-19\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 105s 516ms/step - loss: 4.9204e-04 - accuracy: 0.7889 - val_loss: 0.0023 - val_accuracy: 0.7620 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 87ms/step - loss: 5.6807e-04 - accuracy: 0.7775 - val_loss: 0.0022 - val_accuracy: 0.7411 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 16s 87ms/step - loss: 3.3950e-04 - accuracy: 0.7882 - val_loss: 0.0018 - val_accuracy: 0.7453 - lr: 0.0996
Epoch 4/100
181/181 [==============================] - 16s 87ms/step - loss: 2.2177e-04 - accuracy: 0.7755 - val_loss: 0.0015 - val_accuracy: 0.7286 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 88ms/step - loss: 3.1863e-04 - accuracy: 0.7838 - val_loss: 0.0014 - val_accuracy: 0.7370 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 87ms/step - loss: 2.8231e-04 - accuracy: 0.7743 - val_loss: 0.0012 - val_accuracy: 0.7286 - lr: 0.0976
Epoch 7/100
181/181 [=================

Epoch 51/100
181/181 [==============================] - 16s 87ms/step - loss: 5.7323e-05 - accuracy: 0.7625 - val_loss: 6.8526e-04 - val_accuracy: 0.7182 - lr: 1.0000e-04
Epoch 52/100
181/181 [==============================] - 16s 87ms/step - loss: 9.0340e-05 - accuracy: 0.7611 - val_loss: 6.8503e-04 - val_accuracy: 0.7203 - lr: 1.9856e-04
Epoch 53/100
181/181 [==============================] - 16s 87ms/step - loss: 7.4206e-05 - accuracy: 0.7636 - val_loss: 6.8469e-04 - val_accuracy: 0.7203 - lr: 4.9387e-04
Epoch 54/100
181/181 [==============================] - 16s 87ms/step - loss: 9.1493e-05 - accuracy: 0.7635 - val_loss: 6.8401e-04 - val_accuracy: 0.7182 - lr: 9.8475e-04
Epoch 55/100
181/181 [==============================] - 16s 87ms/step - loss: 8.6596e-05 - accuracy: 0.7605 - val_loss: 6.8331e-04 - val_accuracy: 0.7182 - lr: 0.0017
Epoch 56/100
181/181 [==============================] - 16s 87ms/step - loss: 6.0780e-05 - accuracy: 0.7669 - val_loss: 6.8119e-04 - val_accuracy: 0.

Epoch 100/100
181/181 [==============================] - 16s 87ms/step - loss: 6.2962e-05 - accuracy: 0.7526 - val_loss: 1.5580e-04 - val_accuracy: 0.7119 - lr: 0.0999


In [73]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_19-42-17
Epoch 1/100
  6/181 [..............................] - ETA: 21s - loss: 2.1224e-04 - accuracy: 0.7917WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0836s vs `on_train_batch_end` time: 0.4574s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 5.9832e-05 - accuracy: 0.7509

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-42-17\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-42-17\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 90s 486ms/step - loss: 5.9802e-05 - accuracy: 0.7508 - val_loss: 1.9763e-04 - val_accuracy: 0.7140 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 87ms/step - loss: 8.8578e-05 - accuracy: 0.7472 - val_loss: 1.7106e-04 - val_accuracy: 0.7119 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 16s 87ms/step - loss: 3.8677e-05 - accuracy: 0.7507 - val_loss: 1.7950e-04 - val_accuracy: 0.7140 - lr: 0.0996
Epoch 4/100
180/181 [============================>.] - ETA: 0s - loss: 5.7699e-05 - accuracy: 0.7519

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-42-17\base_ckpts\cp_04.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_19-42-17\base_ckpts\cp_04.ckpt\assets


181/181 [==============================] - 86s 480ms/step - loss: 5.7671e-05 - accuracy: 0.7518 - val_loss: 1.6845e-04 - val_accuracy: 0.7161 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 87ms/step - loss: 8.1977e-05 - accuracy: 0.7535 - val_loss: 1.7592e-04 - val_accuracy: 0.7161 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 86ms/step - loss: 6.7534e-05 - accuracy: 0.7498 - val_loss: 1.7420e-04 - val_accuracy: 0.7161 - lr: 0.0976
Epoch 7/100
181/181 [==============================] - 16s 88ms/step - loss: 5.3435e-05 - accuracy: 0.7459 - val_loss: 2.1020e-04 - val_accuracy: 0.7098 - lr: 0.0965
Epoch 8/100
181/181 [==============================] - 16s 87ms/step - loss: 4.7869e-05 - accuracy: 0.7548 - val_loss: 2.3753e-04 - val_accuracy: 0.7140 - lr: 0.0952
Epoch 9/100
181/181 [==============================] - 16s 87ms/step - loss: 3.1283e-05 - accuracy: 0.7454 - val_loss: 2.3575e-04 - val_accuracy: 0.7098 - lr: 0.0938
Epoch 10/100
18

181/181 [==============================] - 16s 87ms/step - loss: 4.4327e-05 - accuracy: 0.7453 - val_loss: 1.3999e-04 - val_accuracy: 0.7098 - lr: 4.9387e-04
Epoch 54/100
181/181 [==============================] - 16s 87ms/step - loss: 4.8104e-05 - accuracy: 0.7464 - val_loss: 1.3996e-04 - val_accuracy: 0.7098 - lr: 9.8475e-04
Epoch 55/100
181/181 [==============================] - 16s 87ms/step - loss: 9.9767e-05 - accuracy: 0.7466 - val_loss: 1.3991e-04 - val_accuracy: 0.7098 - lr: 0.0017
Epoch 56/100
181/181 [==============================] - 16s 87ms/step - loss: 2.4702e-05 - accuracy: 0.7446 - val_loss: 1.3956e-04 - val_accuracy: 0.7098 - lr: 0.0025
Epoch 57/100
181/181 [==============================] - 16s 87ms/step - loss: 5.8323e-05 - accuracy: 0.7420 - val_loss: 1.3938e-04 - val_accuracy: 0.7098 - lr: 0.0036
Epoch 58/100
181/181 [==============================] - 16s 86ms/step - loss: 3.3622e-05 - accuracy: 0.7402 - val_loss: 1.3901e-04 - val_accuracy: 0.7098 - lr: 0.0049
Epo

In [74]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_20-11-09
Epoch 1/100
  6/181 [..............................] - ETA: 18s - loss: 2.4768e-06 - accuracy: 0.7656WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0776s vs `on_train_batch_end` time: 0.4446s). Check your callbacks.


181/181 [==============================] - ETA: 0s - loss: 2.4398e-05 - accuracy: 0.7329

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-11-09\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-11-09\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 93s 504ms/step - loss: 2.4398e-05 - accuracy: 0.7329 - val_loss: 8.9310e-05 - val_accuracy: 0.7098 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 85ms/step - loss: 1.5605e-05 - accuracy: 0.7375 - val_loss: 8.5520e-05 - val_accuracy: 0.7077 - lr: 0.0999
Epoch 3/100
180/181 [============================>.] - ETA: 0s - loss: 8.5171e-05 - accuracy: 0.7351

...

181/181 [==============================] - 87s 484ms/step - loss: 2.6981e-05 - accuracy: 0.7339 - val_loss: 8.8435e-05 - val_accuracy: 0.7161 - lr: 0.0976
Epoch 7/100
181/181 [==============================] - 16s 85ms/step - loss: 1.6948e-05 - accuracy: 0.7393 - val_loss: 1.0565e-04 - val_accuracy: 0.7119 - lr: 0.0965
Epoch 8/100
181/181 [==============================] - 16s 86ms/step - loss: 1.3211e-05 - accuracy: 0.7355 - val_loss: 1.0532e-04 - val_accuracy: 0.7119 - lr: 0.0952
Epoch 9/100
181/181 [==============================] - 16s 86ms/step - loss: 3.5407e-05 - accuracy: 0.7387 - val_loss: 1.0076e-04 - val_accuracy: 0.7098 - lr: 0.0938
Epoch 10/100
181/181 [==============================] - 16s 87ms/step - loss: 1.9188e-05 - accuracy: 0.7371 - val_loss: 1.0278e-04 - val_accuracy: 0.7161 - lr: 0.0922
Epoch 11/100
181/181 [==============================] - ETA: 0s - loss: 3.5650e-05 - accuracy: 0.7298

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-11-09\base_ckpts\cp_11.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-11-09\base_ckpts\cp_11.ckpt\assets


181/181 [==============================] - 90s 501ms/step - loss: 3.5650e-05 - accuracy: 0.7298 - val_loss: 1.0515e-04 - val_accuracy: 0.7182 - lr: 0.0905
Epoch 12/100
181/181 [==============================] - 16s 85ms/step - loss: 1.8101e-05 - accuracy: 0.7362 - val_loss: 1.0341e-04 - val_accuracy: 0.7119 - lr: 0.0885
Epoch 13/100
181/181 [==============================] - 16s 86ms/step - loss: 1.7391e-05 - accuracy: 0.7314 - val_loss: 9.5322e-05 - val_accuracy: 0.7140 - lr: 0.0865
Epoch 14/100
181/181 [==============================] - 16s 86ms/step - loss: 1.8110e-05 - accuracy: 0.7412 - val_loss: 9.5724e-05 - val_accuracy: 0.7140 - lr: 0.0842
Epoch 15/100
181/181 [==============================] - 16s 87ms/step - loss: 1.6312e-05 - accuracy: 0.7320 - val_loss: 8.8958e-05 - val_accuracy: 0.7119 - lr: 0.0819
Epoch 16/100
181/181 [==============================] - 16s 86ms/step - loss: 8.6010e-06 - accuracy: 0.7270 - val_loss: 9.1411e-05 - val_accuracy: 0.7119 - lr: 0.0794
Epoch 17/1

Epoch 60/100
181/181 [==============================] - 16s 86ms/step - loss: 1.5265e-05 - accuracy: 0.7313 - val_loss: 7.4422e-05 - val_accuracy: 0.6994 - lr: 0.0079
Epoch 61/100
181/181 [==============================] - 16s 86ms/step - loss: 2.1845e-05 - accuracy: 0.7324 - val_loss: 7.6484e-05 - val_accuracy: 0.6994 - lr: 0.0096
Epoch 62/100
181/181 [==============================] - 16s 86ms/step - loss: 2.3578e-05 - accuracy: 0.7326 - val_loss: 7.6309e-05 - val_accuracy: 0.6994 - lr: 0.0116
Epoch 63/100
181/181 [==============================] - 16s 86ms/step - loss: 2.1554e-05 - accuracy: 0.7275 - val_loss: 7.5982e-05 - val_accuracy: 0.7015 - lr: 0.0136
Epoch 64/100
181/181 [==============================] - 16s 86ms/step - loss: 2.5376e-05 - accuracy: 0.7297 - val_loss: 7.4729e-05 - val_accuracy: 0.7015 - lr: 0.0159
Epoch 65/100
181/181 [==============================] - 16s 87ms/step - loss: 2.2249e-05 - accuracy: 0.7279 - val_loss: 7.4818e-05 - val_accuracy: 0.7015 - lr: 0.018

In [75]:
model=tfk.models.load_model("experiments/KF_3.ckpt/base_ckpts/cp_375.ckpt")

In [76]:
model.compile(
            loss=categorical_focal_loss,
            optimizer=tfk.optimizers.SGD(learning_rate=1e-1),
            metrics=['accuracy']
        )

In [77]:
generator=AugmentDataGenerator(X_train_KF[3],y_train_KF[3],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[3],y_val_KF[3],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

foldername = foldermode + '_' + str(date)
print(foldername)

Dec17_20-43-23
ResLSTM_Dec17_20-43-23


In [78]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/100
  6/181 [..............................] - ETA: 19s - loss: 4.1609e-05 - accuracy: 0.8203WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0820s vs `on_train_batch_end` time: 0.4455s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 5.0462e-04 - accuracy: 0.8263

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-43-23\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_20-43-23\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 105s 515ms/step - loss: 5.0441e-04 - accuracy: 0.8263 - val_loss: 0.0175 - val_accuracy: 0.7286 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 86ms/step - loss: 3.1624e-04 - accuracy: 0.8161 - val_loss: 0.0157 - val_accuracy: 0.7203 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 16s 86ms/step - loss: 4.0806e-04 - accuracy: 0.8098 - val_loss: 0.0137 - val_accuracy: 0.7161 - lr: 0.0996
Epoch 4/100
181/181 [==============================] - 16s 87ms/step - loss: 3.4350e-04 - accuracy: 0.8042 - val_loss: 0.0129 - val_accuracy: 0.7015 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 87ms/step - loss: 2.2145e-04 - accuracy: 0.7938 - val_loss: 0.0123 - val_accuracy: 0.6827 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 87ms/step - loss: 3.3899e-04 - accuracy: 0.7996 - val_loss: 0.0107 - val_accuracy: 0.6994 - lr: 0.0976
Epoch 7/100
181/181 [=================

In [79]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_21-11-16
Epoch 1/100
  6/181 [..............................] - ETA: 18s - loss: 5.8070e-06 - accuracy: 0.7552WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0711s vs `on_train_batch_end` time: 0.4493s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 6.4988e-05 - accuracy: 0.7463

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-11-16\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-11-16\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 89s 478ms/step - loss: 6.4960e-05 - accuracy: 0.7462 - val_loss: 0.0037 - val_accuracy: 0.6075 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 85ms/step - loss: 5.8801e-05 - accuracy: 0.7377 - val_loss: 0.0035 - val_accuracy: 0.6075 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 16s 86ms/step - loss: 2.5525e-05 - accuracy: 0.7339 - val_loss: 0.0035 - val_accuracy: 0.6075 - lr: 0.0996
Epoch 4/100
180/181 [============================>.] - ETA: 0s - loss: 5.9484e-05 - accuracy: 0.7327

...

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-11-16\base_ckpts\cp_11.ckpt\assets


181/181 [==============================] - 91s 504ms/step - loss: 4.6071e-05 - accuracy: 0.7267 - val_loss: 0.0035 - val_accuracy: 0.6159 - lr: 0.0905
Epoch 12/100
181/181 [==============================] - 16s 86ms/step - loss: 9.0082e-05 - accuracy: 0.7399 - val_loss: 0.0034 - val_accuracy: 0.6138 - lr: 0.0885
Epoch 13/100
181/181 [==============================] - 16s 86ms/step - loss: 7.7311e-05 - accuracy: 0.7360 - val_loss: 0.0030 - val_accuracy: 0.6075 - lr: 0.0865
Epoch 14/100
181/181 [==============================] - 16s 86ms/step - loss: 8.3794e-05 - accuracy: 0.7411 - val_loss: 0.0032 - val_accuracy: 0.6138 - lr: 0.0842
Epoch 15/100
181/181 [==============================] - 16s 86ms/step - loss: 9.2215e-05 - accuracy: 0.7387 - val_loss: 0.0031 - val_accuracy: 0.6075 - lr: 0.0819
Epoch 16/100
181/181 [==============================] - 16s 86ms/step - loss: 3.7922e-05 - accuracy: 0.7401 - val_loss: 0.0030 - val_accuracy: 0.6117 - lr: 0.0794
Epoch 17/100
181/181 [============

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-11-16\base_ckpts\cp_22.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-11-16\base_ckpts\cp_22.ckpt\assets


181/181 [==============================] - 85s 474ms/step - loss: 6.2511e-05 - accuracy: 0.7362 - val_loss: 0.0035 - val_accuracy: 0.6200 - lr: 0.0625
Epoch 23/100
181/181 [==============================] - 16s 86ms/step - loss: 4.9695e-05 - accuracy: 0.7359 - val_loss: 0.0034 - val_accuracy: 0.6117 - lr: 0.0594
Epoch 24/100
181/181 [==============================] - 16s 87ms/step - loss: 3.3700e-05 - accuracy: 0.7405 - val_loss: 0.0035 - val_accuracy: 0.6138 - lr: 0.0563
Epoch 25/100
181/181 [==============================] - 16s 87ms/step - loss: 1.6521e-05 - accuracy: 0.7349 - val_loss: 0.0035 - val_accuracy: 0.6138 - lr: 0.0532
Epoch 26/100
181/181 [==============================] - 16s 86ms/step - loss: 3.8310e-05 - accuracy: 0.7479 - val_loss: 0.0035 - val_accuracy: 0.6138 - lr: 0.0501
Epoch 27/100
181/181 [==============================] - 16s 87ms/step - loss: 3.5761e-05 - accuracy: 0.7440 - val_loss: 0.0035 - val_accuracy: 0.6138 - lr: 0.0469
Epoch 28/100
181/181 [============

In [80]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec17_21-44-49
Epoch 1/100
  6/181 [..............................] - ETA: 18s - loss: 1.0515e-05 - accuracy: 0.7083WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0736s vs `on_train_batch_end` time: 0.4440s). Check your callbacks.


180/181 [============================>.] - ETA: 0s - loss: 4.2641e-05 - accuracy: 0.7216

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-44-49\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-44-49\base_ckpts\cp_01.ckpt\assets


181/181 [==============================] - 94s 507ms/step - loss: 4.2623e-05 - accuracy: 0.7216 - val_loss: 0.0029 - val_accuracy: 0.5825 - lr: 0.1000
Epoch 2/100
181/181 [==============================] - 16s 85ms/step - loss: 3.9227e-05 - accuracy: 0.7257 - val_loss: 0.0027 - val_accuracy: 0.5741 - lr: 0.0999
Epoch 3/100
181/181 [==============================] - 15s 85ms/step - loss: 3.6104e-05 - accuracy: 0.7236 - val_loss: 0.0027 - val_accuracy: 0.5678 - lr: 0.0996
Epoch 4/100
181/181 [==============================] - 15s 85ms/step - loss: 3.7245e-05 - accuracy: 0.7276 - val_loss: 0.0028 - val_accuracy: 0.5699 - lr: 0.0991
Epoch 5/100
181/181 [==============================] - 16s 86ms/step - loss: 2.4905e-05 - accuracy: 0.7196 - val_loss: 0.0028 - val_accuracy: 0.5637 - lr: 0.0984
Epoch 6/100
181/181 [==============================] - 16s 85ms/step - loss: 3.6200e-05 - accuracy: 0.7210 - val_loss: 0.0028 - val_accuracy: 0.5699 - lr: 0.0976
Epoch 7/100
181/181 [==================

...

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-44-49\base_ckpts\cp_24.ckpt\assets


181/181 [==============================] - 86s 475ms/step - loss: 7.9939e-05 - accuracy: 0.7216 - val_loss: 0.0026 - val_accuracy: 0.5971 - lr: 0.0563
Epoch 25/100
181/181 [==============================] - 16s 87ms/step - loss: 3.7329e-05 - accuracy: 0.7134 - val_loss: 0.0026 - val_accuracy: 0.5971 - lr: 0.0532
Epoch 26/100
181/181 [==============================] - 16s 86ms/step - loss: 4.8458e-05 - accuracy: 0.7156 - val_loss: 0.0026 - val_accuracy: 0.5950 - lr: 0.0501
Epoch 27/100
181/181 [==============================] - 16s 86ms/step - loss: 2.2508e-05 - accuracy: 0.7149 - val_loss: 0.0025 - val_accuracy: 0.5908 - lr: 0.0469
Epoch 28/100
181/181 [==============================] - 16s 87ms/step - loss: 8.8884e-06 - accuracy: 0.7216 - val_loss: 0.0025 - val_accuracy: 0.5908 - lr: 0.0438
Epoch 29/100
181/181 [==============================] - 16s 87ms/step - loss: 1.7979e-05 - accuracy: 0.7212 - val_loss: 0.0025 - val_accuracy: 0.5950 - lr: 0.0407
Epoch 30/100
181/181 [============

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-44-49\base_ckpts\cp_31.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec17_21-44-49\base_ckpts\cp_31.ckpt\assets


181/181 [==============================] - 85s 470ms/step - loss: 8.9459e-05 - accuracy: 0.7164 - val_loss: 0.0025 - val_accuracy: 0.5992 - lr: 0.0346
Epoch 32/100
181/181 [==============================] - 16s 86ms/step - loss: 1.8249e-05 - accuracy: 0.7160 - val_loss: 0.0025 - val_accuracy: 0.5971 - lr: 0.0317
Epoch 33/100
181/181 [==============================] - 16s 86ms/step - loss: 2.4980e-05 - accuracy: 0.7177 - val_loss: 0.0024 - val_accuracy: 0.5908 - lr: 0.0288
Epoch 34/100
181/181 [==============================] - 16s 86ms/step - loss: 2.5607e-05 - accuracy: 0.7187 - val_loss: 0.0024 - val_accuracy: 0.5908 - lr: 0.0260
Epoch 35/100
181/181 [==============================] - 16s 86ms/step - loss: 2.7452e-05 - accuracy: 0.7160 - val_loss: 0.0024 - val_accuracy: 0.5908 - lr: 0.0233
Epoch 36/100
181/181 [==============================] - 16s 86ms/step - loss: 2.4842e-05 - accuracy: 0.7111 - val_loss: 0.0024 - val_accuracy: 0.5929 - lr: 0.0207
Epoch 37/100
181/181 [============

In [28]:
model=tfk.models.load_model("experiments/KF_4.ckpt/base_ckpts/cp_351.ckpt")

In [37]:
model.compile(
            loss=categorical_focal_loss,
            optimizer=tfk.optimizers.SGD(learning_rate=1e-1),
            metrics=['accuracy']
        )

In [38]:
generator=AugmentDataGenerator(X_train_KF[4],y_train_KF[4],p=0.5)
val_generator=AugmentDataGenerator(X_val_KF[4],y_val_KF[4],p=0)
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))

foldername = foldermode + '_' + str(date)
print(foldername)

Dec18_02-38-29
ResLSTM_Dec18_02-38-29


In [39]:
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Epoch 1/100
181/182 [============================>.] - ETA: 0s - loss: 3.9546e-04 - accuracy: 0.8218

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_02-38-29\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_02-38-29\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 100s 475ms/step - loss: 3.9520e-04 - accuracy: 0.8217 - val_loss: 0.0082 - val_accuracy: 0.7495 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 14s 76ms/step - loss: 4.0105e-04 - accuracy: 0.8131 - val_loss: 0.0074 - val_accuracy: 0.7453 - lr: 0.0999
Epoch 3/100
182/182 [==============================] - 14s 77ms/step - loss: 4.2298e-04 - accuracy: 0.7993 - val_loss: 0.0069 - val_accuracy: 0.7495 - lr: 0.0996
Epoch 4/100
182/182 [==============================] - 14s 77ms/step - loss: 3.7660e-04 - accuracy: 0.8009 - val_loss: 0.0064 - val_accuracy: 0.7474 - lr: 0.0991
Epoch 5/100
182/182 [==============================] - 14s 78ms/step - loss: 2.9111e-04 - accuracy: 0.7990 - val_loss: 0.0065 - val_accuracy: 0.7390 - lr: 0.0984
Epoch 6/100
182/182 [==============================] - 14s 78ms/step - loss: 3.8133e-04 - accuracy: 0.7993 - val_loss: 0.0061 - val_accuracy: 0.7432 - lr: 0.0976
Epoch 7/100
182/182 [=================

In [40]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec18_03-03-42
Epoch 1/100
  6/182 [..............................] - ETA: 17s - loss: 1.1801e-05 - accuracy: 0.7474WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0661s vs `on_train_batch_end` time: 0.4368s). Check your callbacks.


181/182 [============================>.] - ETA: 0s - loss: 7.8230e-05 - accuracy: 0.7611

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-03-42\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-03-42\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 89s 475ms/step - loss: 7.8176e-05 - accuracy: 0.7613 - val_loss: 0.0015 - val_accuracy: 0.6994 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 14s 78ms/step - loss: 9.7659e-05 - accuracy: 0.7566 - val_loss: 0.0014 - val_accuracy: 0.6931 - lr: 0.0999
Epoch 3/100
182/182 [==============================] - 14s 78ms/step - loss: 5.0250e-05 - accuracy: 0.7597 - val_loss: 0.0014 - val_accuracy: 0.6827 - lr: 0.0996
Epoch 4/100
182/182 [==============================] - 14s 77ms/step - loss: 4.1831e-05 - accuracy: 0.7517 - val_loss: 0.0014 - val_accuracy: 0.6889 - lr: 0.0991
Epoch 5/100
182/182 [==============================] - 14s 78ms/step - loss: 5.8207e-05 - accuracy: 0.7547 - val_loss: 0.0013 - val_accuracy: 0.6889 - lr: 0.0984
Epoch 6/100
182/182 [==============================] - 14s 78ms/step - loss: 6.1643e-05 - accuracy: 0.7604 - val_loss: 0.0013 - val_accuracy: 0.6889 - lr: 0.0976
Epoch 7/100
182/182 [==================

Epoch 51/100
182/182 [==============================] - 14s 78ms/step - loss: 1.1839e-04 - accuracy: 0.7553 - val_loss: 9.6579e-04 - val_accuracy: 0.6764 - lr: 1.0000e-04
Epoch 52/100
182/182 [==============================] - 14s 78ms/step - loss: 4.4318e-05 - accuracy: 0.7547 - val_loss: 9.6595e-04 - val_accuracy: 0.6764 - lr: 1.9856e-04
Epoch 53/100
182/182 [==============================] - 14s 78ms/step - loss: 3.6903e-05 - accuracy: 0.7537 - val_loss: 9.6571e-04 - val_accuracy: 0.6764 - lr: 4.9387e-04
Epoch 54/100
182/182 [==============================] - 14s 78ms/step - loss: 8.1089e-05 - accuracy: 0.7509 - val_loss: 9.6587e-04 - val_accuracy: 0.6764 - lr: 9.8475e-04
Epoch 55/100
182/182 [==============================] - 14s 79ms/step - loss: 5.3752e-05 - accuracy: 0.7562 - val_loss: 9.6546e-04 - val_accuracy: 0.6764 - lr: 0.0017
Epoch 56/100
182/182 [==============================] - 14s 78ms/step - loss: 2.3934e-05 - accuracy: 0.7560 - val_loss: 9.6570e-04 - val_accuracy: 0.

Epoch 100/100
182/182 [==============================] - 14s 78ms/step - loss: 1.7464e-05 - accuracy: 0.7480 - val_loss: 7.1000e-04 - val_accuracy: 0.6743 - lr: 0.0999


In [41]:
callbacks, date = create_folders_and_callbacks(foldermode)
callbacks.append(CosineAnnealingScheduler(T_max=50, eta_max=1e-1, eta_min=1e-4))
history = model.fit(
    generator,
    validation_data = val_generator,
    epochs = 100,
    shuffle=False,
    callbacks=callbacks
).history

Dec18_03-28-42
Epoch 1/100
  6/182 [..............................] - ETA: 18s - loss: 1.2208e-05 - accuracy: 0.7526WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0707s vs `on_train_batch_end` time: 0.4378s). Check your callbacks.


181/182 [============================>.] - ETA: 0s - loss: 9.3682e-05 - accuracy: 0.7514

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-28-42\base_ckpts\cp_01.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-28-42\base_ckpts\cp_01.ckpt\assets


182/182 [==============================] - 87s 465ms/step - loss: 9.3622e-05 - accuracy: 0.7513 - val_loss: 7.1791e-04 - val_accuracy: 0.6764 - lr: 0.1000
Epoch 2/100
182/182 [==============================] - 15s 78ms/step - loss: 3.2666e-05 - accuracy: 0.7506 - val_loss: 7.2569e-04 - val_accuracy: 0.6743 - lr: 0.0999
Epoch 3/100
181/182 [============================>.] - ETA: 0s - loss: 4.1507e-05 - accuracy: 0.7416

...

182/182 [==============================] - 84s 462ms/step - loss: 2.6544e-05 - accuracy: 0.7524 - val_loss: 6.8343e-04 - val_accuracy: 0.6848 - lr: 0.0984
Epoch 6/100
181/182 [============================>.] - ETA: 0s - loss: 3.8208e-05 - accuracy: 0.7451

INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-28-42\base_ckpts\cp_06.ckpt\assets


INFO:tensorflow:Assets written to: experiments\ResLSTM_Dec18_03-28-42\base_ckpts\cp_06.ckpt\assets


182/182 [==============================] - 85s 467ms/step - loss: 3.8182e-05 - accuracy: 0.7451 - val_loss: 6.8615e-04 - val_accuracy: 0.6889 - lr: 0.0976
Epoch 7/100
182/182 [==============================] - 15s 78ms/step - loss: 2.2939e-05 - accuracy: 0.7478 - val_loss: 6.8967e-04 - val_accuracy: 0.6848 - lr: 0.0965
Epoch 8/100
182/182 [==============================] - 14s 78ms/step - loss: 2.0387e-05 - accuracy: 0.7567 - val_loss: 6.8715e-04 - val_accuracy: 0.6806 - lr: 0.0952
Epoch 9/100
182/182 [==============================] - 14s 78ms/step - loss: 1.7150e-05 - accuracy: 0.7456 - val_loss: 6.7699e-04 - val_accuracy: 0.6827 - lr: 0.0938
Epoch 10/100
182/182 [==============================] - 14s 77ms/step - loss: 3.1392e-05 - accuracy: 0.7508 - val_loss: 6.6993e-04 - val_accuracy: 0.6868 - lr: 0.0922
Epoch 11/100
182/182 [==============================] - 14s 78ms/step - loss: 7.1519e-05 - accuracy: 0.7552 - val_loss: 6.7355e-04 - val_accuracy: 0.6827 - lr: 0.0905
Epoch 12/100


Epoch 55/100
182/182 [==============================] - 14s 79ms/step - loss: 2.9659e-05 - accuracy: 0.7346 - val_loss: 5.8720e-04 - val_accuracy: 0.6722 - lr: 0.0017
Epoch 56/100
182/182 [==============================] - 14s 78ms/step - loss: 2.3461e-05 - accuracy: 0.7476 - val_loss: 5.8582e-04 - val_accuracy: 0.6722 - lr: 0.0025
Epoch 57/100
182/182 [==============================] - 14s 78ms/step - loss: 1.6157e-05 - accuracy: 0.7431 - val_loss: 5.8582e-04 - val_accuracy: 0.6722 - lr: 0.0036
Epoch 58/100
182/182 [==============================] - 14s 79ms/step - loss: 2.6974e-05 - accuracy: 0.7421 - val_loss: 5.8549e-04 - val_accuracy: 0.6722 - lr: 0.0049
Epoch 59/100
182/182 [==============================] - 14s 78ms/step - loss: 2.6756e-05 - accuracy: 0.7454 - val_loss: 5.8532e-04 - val_accuracy: 0.6722 - lr: 0.0063
Epoch 60/100
182/182 [==============================] - 14s 78ms/step - loss: 3.3837e-05 - accuracy: 0.7355 - val_loss: 5.7960e-04 - val_accuracy: 0.6722 - lr: 0.007

# Model saving
The model is saved and it can be reloaded if necessary.

In [ ]:
# Save the model
model.save(mode)